# Train a classification model with Automated Machine Learning

###### 1. Check the latest version of the azureml-ai-ml package 

In [ ]:
pip show azure-ai-ml

###### 2. Connect to workspace

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

# To connect to a workspace, we need identifier parameters
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

###### 3. Prepare data

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

###### 4. Configure automated machine learning job

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",                                  # Compute cluster used: aml-cluster
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",                          # Sets Diabetic as the target column
    primary_metric="accuracy",                              # Sets accuracy as the primary metric
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60,                                     # Times out after 60 minutes of total training time
    trial_timeout_minutes=20, 
    max_trials=5,                                           # Trains a maximum of 5 models
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"],     # No model will be trained with the LogisticRegression algorithm
    enable_onnx_compatible_models=True
)

###### 5. Run an automated machine learning job

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)